<a href="https://colab.research.google.com/github/JaimRM/Portfolio-Management/blob/main/backtest_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install backtrader
!pip install backtrader[plotting] # Adds Matplotlib for charts
!pip install yfinance # A handy library for getting data

In [27]:
from datetime import datetime
import backtrader as bt
import yfinance as yf
import pandas as pd # Import pandas for MultiIndex check
import matplotlib.pyplot as plt # Import matplotlib for plotting

# --- 1. Define the Strategy ---
class SmaCross(bt.Strategy):
    # Parameters are great for optimization later on!
    params = (
        ('pfast', 10),  # Period for the fast moving average (SMA)
        ('pslow', 30)   # Period for the slow moving average (SMA)
    )

    def __init__(self):
        # Keep a reference to the close price line
        self.dataclose = self.datas[0].close
        # Keep track of pending orders
        self.order = None

        # Create the SMAs
        sma1 = bt.indicators.SimpleMovingAverage(self.dataclose, period=self.p.pfast)
        sma2 = bt.indicators.SimpleMovingAverage(self.dataclose, period=self.p.pslow)

        # Calculate the crossover signal
        self.crossover = bt.indicators.CrossOver(sma1, sma2)

    def notify_order(self, order):
        # Called when an order's status changes
        if order.status in [order.Submitted, order.Accepted]:
            # Order has been submitted/accepted - no action required
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                # self.log(f'BUY EXECUTED, Price: {order.executed.price:,.2f}, Comm: {order.executed.comm:,.2f}')
                pass # Only for verbose logging
            elif order.issell():
                # self.log(f'SELL EXECUTED, Price: {order.executed.price:,.2f}, Comm: {order.executed.comm:,.2f}')
                pass # Only for verbose logging
            self.order = None # Reset order status

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        # This is the main logic function, called on every new bar (e.g., every day)

        # Simply return if there is a pending order
        if self.order:
            return

        # Check if we ARE NOT in the market
        if not self.position:
            # Crossover > 0 means the fast SMA crossed ABOVE the slow SMA (BUY SIGNAL)
            if self.crossover > 0:
                # self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy(size=100) # Enter a long position of 100 shares

        # Check if we ARE in the market (we have a long position)
        elif self.position.size > 0:
            # Crossover < 0 means the fast SMA crossed BELOW the slow SMA (SELL SIGNAL)
            if self.crossover < 0:
                # self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.close() # Close the current position

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')


# --- 2. Main Backtesting Execution ---
if __name__ == '__main__':
    # Initialize the Cerebro engine
    cerebro = bt.Cerebro()

    # Add the strategy with optimization parameters
    cerebro.optstrategy(
        SmaCross,
        pfast=range(10, 31, 5),  # Fast SMA periods from 10 to 30, step 5
        pslow=range(30, 61, 5)   # Slow SMA periods from 30 to 60, step 5
    )

    # Get data from Yahoo Finance (using yfinance built-in functionality)
    # Define a data range for a good backtest period
    df_data = yf.download(
        'AAPL',
        start=datetime(2020, 1, 1),
        end=datetime(2024, 1, 1)
    )

    # Check if columns are MultiIndex and flatten them
    if isinstance(df_data.columns, pd.MultiIndex):
        # Droplevel(1) will remove the 'AAPL' part from column names like ('Open', 'AAPL')
        df_data.columns = df_data.columns.droplevel(1)

    # Create the Backtrader data feed from the potentially flattened DataFrame
    data = bt.feeds.PandasData(
        dataname=df_data
    )

    # Add the data to Cerebro
    cerebro.adddata(data)

    # Set initial capital, commission, and position sizer
    initial_cash = 10000.0
    cerebro.broker.setcash(initial_cash)
    cerebro.broker.setcommission(commission=0.001) # 0.1% commission

    # Add a key analyzer for professional reporting
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    # Add a returns analyzer to get final value if needed, though we will re-run the best strategy.
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')


    # Print out the starting conditions
    print(f'Starting Portfolio Value: {cerebro.broker.getvalue():,.2f}')

    # Run the backtest!
    results = cerebro.run()

    # After optimization, 'results' is a list of lists, where each inner list contains one
    # OptReturn object for each strategy run. We need to iterate through them.
    print('\nOptimization Results:')
    best_opt_return = None # This will store the OptReturn object for the best strategy
    best_sharpe = -float('inf') # Initialize with a very low sharpe ratio

    for rlist in results:
        for strat_opt_return in rlist: # Rename 'strat' to 'strat_opt_return' for clarity
            pfast_val = strat_opt_return.p.pfast
            pslow_val = strat_opt_return.p.pslow

            # Safely get Sharpe Ratio
            sharpe = 0.0 # Default value
            sharpe_analyzer_result = strat_opt_return.analyzers.sharpe.get_analysis()
            if sharpe_analyzer_result:
                sharpe_candidate = sharpe_analyzer_result.get('sharperatio')
                if sharpe_candidate is not None:
                    try:
                        sharpe = float(sharpe_candidate)
                        if pd.isna(sharpe):
                            sharpe = 0.0
                    except (ValueError, TypeError):
                        sharpe = 0.0

            # Safely get Returns
            rtot_val = 0.0 # Default value
            returns_analyzer_result = strat_opt_return.analyzers.returns.get_analysis()
            if returns_analyzer_result:
                rtot_candidate = returns_analyzer_result.get('rtot')
                if rtot_candidate is not None:
                    try:
                        rtot_val = float(rtot_candidate)
                        if pd.isna(rtot_val):
                            rtot_val = 0.0
                    except (ValueError, TypeError):
                        rtot_val = 0.0

            # Calculate final value.
            final_value_for_log = initial_cash * (1 + rtot_val)
            if pd.isna(final_value_for_log): # Fallback for NaN in calculation
                final_value_for_log = initial_cash

            print(f'  Params: pfast={pfast_val}, pslow={pslow_val} -> Final Value: {final_value_for_log:,.2f}, Sharpe Ratio: {sharpe:.4f}')

            if sharpe > best_sharpe:
                best_sharpe = sharpe
                best_opt_return = strat_opt_return # Store the OptReturn object

    if best_opt_return:
        # Get parameters from the best OptReturn object
        optimal_pfast = best_opt_return.p.pfast
        optimal_pslow = best_opt_return.p.pslow

        print(f'\nBest Strategy Parameters Found:')
        print(f'  pfast: {optimal_pfast}')
        print(f'  pslow: {optimal_pslow}')
        print(f'  Sharpe Ratio: {best_sharpe:.4f}')

        # --- Re-run the best strategy for plotting and final value ---
        print('\nRe-running and plotting best strategy results...')
        cerebro_plot = bt.Cerebro()
        cerebro_plot.addstrategy(SmaCross, pfast=optimal_pfast, pslow=optimal_pslow)

        # Re-create the data feed as it's consumed by the first cerebro.run()
        data_plot = bt.feeds.PandasData(
            dataname=df_data
        )
        cerebro_plot.adddata(data_plot)

        cerebro_plot.broker.setcash(initial_cash)
        cerebro_plot.broker.setcommission(commission=0.001)
        cerebro_plot.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe') # Add analyzer for plotting run too
        cerebro_plot.addanalyzer(bt.analyzers.Returns, _name='returns')


        strategies_plot = cerebro_plot.run()
        # Get the single strategy instance from the plot run
        best_strategy_instance = strategies_plot[0]

        # Extract final portfolio value from the re-run strategy
        final_portfolio_value_best_strategy = best_strategy_instance.broker.getvalue()

        # Get Sharpe Ratio from the re-run strategy (should match best_sharpe)
        sharpe_ratio_best_strategy_plot_run = best_strategy_instance.analyzers.sharpe.get_analysis().get('sharperatio', 0.0)

        print(f'  Final Portfolio Value (from optimal strategy run): {final_portfolio_value_best_strategy:,.2f}')
        print(f'  Sharpe Ratio (from optimal strategy run): {sharpe_ratio_best_strategy_plot_run:.4f}')


        cerebro_plot.plot()
        plt.show()
    else:
        print('No strategies run or no best strategy found.')

/tmp/ipython-input-2931681198.py:88: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_data = yf.download(
[*********************100%***********************]  1 of 1 completed


Starting Portfolio Value: 10,000.00
Order Canceled/Margin/Rejected
Order Canceled/Margin/RejectedOrder Canceled/Margin/Rejected

Order Canceled/Margin/RejectedOrder Canceled/Margin/Rejected

Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/RejectedOrder Canceled/Margin/Rejected

Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Order Canceled/Margin/Rejected
Ord